# Generate text with RuGPTs in huggingface
How to generate text with pretrained RuGPTs models with huggingface.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [1]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


If code below doesn't work, check your cuda version and installation here https://pytorch.org/get-started/previous-versions/

In [ ]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip3 install transformers==3.5.0

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

## Generate

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()


def generate(
    model, tok, text,
    do_sample=True, max_length=50, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

### RuGPT2Large

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt2large")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)
generated[0]

### RuGPT3Small

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3small_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

In [ ]:
generated[0]

'Александр Сергеевич Пушкин родился в  1824 г. в семье поэта и драматурга Александра Сергеевича Пушкина (1825-1894) и Марии Александровны Пушкиной (1828-1897). В 17 лет он поступил на юридический факультет Московского'

### RuGPT3Medium

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3medium_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

In [ ]:
generated[0]

'Александр Сергеевич Пушкин родился в  1799 году. Его отец, Александр Сергеевич Пушкин, был генерал-адъютантом и воспитателем великого поэта Александра Сергеевича Грибоедова (1795–1829). В 1811 году он поступил'

### RuGPT3Large

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3large_based_on_gpt2")
generated = generate(model, tok, "Александр Сергеевич Пушкин родился в ", num_beams=10)

In [ ]:
generated[0]

'Александр Сергеевич Пушкин родился в \n1799 году. Его отец был крепостным крестьянином, а мать – крепостной крестьянкой. Детство и юность Пушкина прошли в деревне Михайловское под Петербургом. В 1820-х годах семья переехала'